In [ ]:
%matplotlib inline

In [ ]:
import sys,os,importlib

### Manually set paths for now

In [ ]:
package_dirs=[os.path.join(os.pardir,'getdatatestbed'),os.pardir]
yaml_dir=os.path.join(os.pardir,'yamls')
output_dir=os.path.join(os.getenv('HOME'),'Public','code','dunex_sprint','products')
datacache=os.path.join(os.getenv('HOME'),'Public','code','dunex_sprint','datacache')
server='FRF'

In [ ]:
global_yaml = os.path.join(yaml_dir,'IntegratedBathyTopo_Global.yml')
var_yaml = os.path.join(yaml_dir,'IntegratedBathyTopo_grid_var.yml')


In [ ]:
for dd in package_dirs:
    if dd not in sys.path:
        print('appending {0} to system path for python import'.format(dd))
        sys.path.append(dd)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import scipy
from scipy import interpolate
import pickle
import netCDF4
import bathyTopoUtils as dut 
import testbedutils
import testbedutils.py2netCDF as py2netCDF
import testbedutils.geoprocess as geoprocess

In [ ]:
from ipywidgets import interact

### Data
Survey transects
https://chldata.erdc.dren.mil/thredds/catalog/frf/geomorphology/elevationTransects/survey/catalog.html

Dune lidar
https://chldata.erdc.dren.mil/thredds/catalog/frf/geomorphology/DEMs/pierLidarDEM/catalog.html

Pier lidar
https://chldata.erdc.dren.mil/thredds/catalog/frf/geomorphology/DEMs/pierLidarDEM/catalog.html


## Steps:
- Generate target grid for interpolation
- Pick a time: grab survey, pier and dune lidar data sets using getFRFdata
- interpolate everything for that day to the target grid, using constant extension in alongshore, see what you get in the cross-shore
- Update daily with irregular new observations



## Requirements, needs
- We want to weight topo more than survey because of accuracy
- We want to weight more recent values over older values
- Topo will be multiple times per day,
- We could try and grab low-tide values, or just grab all the values

## TODO
- refactor main loop to go from date range to netcdf
- write netcdf output file for bathy
- figure out time conventions for running: day1 0 hours to day1+24 hours?
- add diagnostic plots
- add cross-validation points for calculating accuracy

### Target grid resolution
0-1400 in along shore, yFRF, 5 m resolution
0-1100 in xFRF, 5 m resolution


In [ ]:
xlim=(0,1100.)
ylim=(0,1400.)
dx=(5.,5.)


In [ ]:
xx=np.arange(xlim[0],xlim[1],dx[0])
yy=np.arange(ylim[0],ylim[1],dx[1])
nx,ny=xx.size,yy.size
XX,YY=np.meshgrid(xx,yy)
assert XX.shape==(ny,nx)
assert YY.shape==(ny,nx)

## Time window

In [ ]:
d2=dt.datetime(year=2021,month=8,day=24)
d1=d2-dt.timedelta(days=1)

In [ ]:
print('Trying to run from {0} to {1}'.format(d1.date(),d2.date()))

In [ ]:
import getDataFRF

In [ ]:
importlib.reload(getDataFRF)
go = getDataFRF.getObs(d1,d2,server=server)

In [ ]:
cross_check_fraction=0.05
check_error=True

In [ ]:
interp_method='linear'

## Load Dune and Pier lidar

In [ ]:
repull=False
if repull:
    topo_dune = go.getLidarDEM(lidarLoc='dune')
    pickle.dump(topo_dune, open(os.path.join(datacache, "topo_dune_{0}.p".format(d1.date())), "wb" ) )


In [ ]:
if 'topo_dune' not in globals():
    topo_dune=pickle.load(open(os.path.join(datacache,"topo_dune_{0}.p".format(d1.date())),"rb"))

Topo elevation shape is ntimes, ny, nx

In [ ]:
if topo_dune['elevation'].ndim==2: #must be 3d
    topo_dune['elevation']=topo_dune['elevation'][np.newaxis,:,:]


In [ ]:
topo_dune.keys()

In [ ]:
print('nx,ny for dune lidar = ({0},{1})'.format(topo_dune['xFRF'].shape[0],topo_dune['yFRF'].shape[0]))
print('xFRF range for dune lidar = ({0},{1})'.format(topo_dune['xFRF'].min(),topo_dune['xFRF'].max()))
print('yFRF range for dune lidar = ({0},{1})'.format(topo_dune['yFRF'].min(),topo_dune['yFRF'].max()))

In [ ]:
print('nt,nx,ny for dune lidar = ({0},{1},{2})'.format(topo_dune['elevation'].shape[0],
                                                   topo_dune['elevation'].shape[1],topo_dune['elevation'].shape[2]))

In [ ]:
Xdune,Ydune=np.meshgrid(topo_dune['xFRF'],topo_dune['yFRF'])

In [ ]:
points_dune=np.vstack((Xdune.flat[:],Ydune.flat[:])).T

In [ ]:
def plot_lidar(X,Y,ztimes,it):
    plt.pcolor(X,Y,ztimes[it],shading='auto')
    plt.colorbar()
def scatter_lidar(X,Y,ztimes,it):
    plt.scatter(X,Y,c=ztimes[it].flat[:])
    plt.xlim([X.min(),X.max()])
    plt.ylim([Y.min(),Y.max()])
    plt.colorbar()    

In [ ]:
interact(lambda it: plot_lidar(Xdune,Ydune,topo_dune['elevation'],it), it=(0,topo_dune['elevation'].shape[0]-1))

In [ ]:
interact(lambda it: scatter_lidar(points_dune[:,0],points_dune[:,1],topo_dune['elevation'],it), it=(0,topo_dune['elevation'].shape[0]-1))

In [ ]:
repull=False
if repull:
    topo_pier = go.getLidarDEM(lidarLoc='pier')
    pickle.dump(topo_pier, open(os.path.join(datacache,"topo_pier_{0}.p".format(d1.date())), "wb" ) )

In [ ]:
if 'topo_pier' not in globals():
    topo_pier=pickle.load(open(os.path.join(datacache,"topo_pier_{0}.p".format(d1.date())),"rb"))

In [ ]:
if topo_pier['elevation'].ndim==2: #must be 3d
    topo_pier['elevation']=topo_pier['elevation'][np.newaxis,:,:]


In [ ]:
print('nx,ny for pier lidar = ({0},{1})'.format(topo_pier['xFRF'].shape[0],topo_pier['yFRF'].shape[0]))
print('xFRF range for pier lidar = ({0},{1})'.format(topo_pier['xFRF'].min(),topo_pier['xFRF'].max()))
print('yFRF range for pier lidar = ({0},{1})'.format(topo_pier['yFRF'].min(),topo_pier['yFRF'].max()))

In [ ]:
print('nt,nx,ny for topo lidar = ({0},{1},{2})'.format(topo_pier['elevation'].shape[0],
                                                       topo_pier['elevation'].shape[1],topo_pier['elevation'].shape[2]))

In [ ]:
Xpier,Ypier=np.meshgrid(topo_pier['xFRF'],topo_pier['yFRF'])

In [ ]:
points_pier=np.vstack((Xpier.flat[:],Ypier.flat[:])).T

In [ ]:
interact(lambda it: plot_lidar(Xpier,Ypier,topo_pier['elevation'],it), it=(0,topo_pier['elevation'].shape[0]-1))

In [ ]:
interact(lambda it: scatter_lidar(points_pier[:,0],points_pier[:,1],topo_pier['elevation'],it), it=(0,topo_pier['elevation'].shape[0]-1))

## Interpoloate Dune and Pier lidar to target grid

### Do one timestep

In [ ]:
dune_values0,dune_points0,Z_dune0=dut.interpolate_masked_lidar(Xdune,Ydune,topo_dune['elevation'][0],XX,YY,method=interp_method)


In [ ]:
nt_dune=topo_dune['elevation'].shape[0]
dune_values,dune_points,Z_dune=dut.interpolate_masked_lidar(np.tile(Xdune,(nt_dune,1,1)),
                                                        np.tile(Ydune,(nt_dune,1,1)),
                                                        topo_dune['elevation'],XX,YY,method=interp_method)

In [ ]:
plt.pcolor(XX,YY,Z_dune,shading='auto')

One time step

In [ ]:
pier_values0,pier_points0,Z_pier0=dut.interpolate_masked_lidar(Xpier,Ypier,topo_pier['elevation'][0],XX,YY,method=interp_method)

In [ ]:
plt.pcolor(XX,YY,Z_pier0,shading='auto')

In [ ]:
nt_pier=topo_pier['elevation'].shape[0]
pier_values,pier_points,Z_pier=dut.interpolate_masked_lidar(np.tile(Xpier,(nt_pier,1,1)),
                                                                np.tile(Ypier,(nt_pier,1,1)),topo_pier['elevation'],XX,YY,method='linear')

In [ ]:
time_slice=0
all_points1,all_values1,Z_all1=dut.combine_and_interpolate_masked_lidar([Xdune,Xpier],[Ydune,Ypier],
                                                                 [topo_dune['elevation'][time_slice],
                                                                  topo_pier['elevation'][time_slice]],
                                                                 XX,YY,method='linear')

In [ ]:
all_points,all_values,Z_all=dut.combine_and_interpolate_masked_lidar([np.tile(Xdune,(nt_dune,1,1)),
                                                                  np.tile(Xpier,(nt_pier,1,1))],
                                                                 [np.tile(Ydune,(nt_dune,1,1)),
                                                                  np.tile(Ypier,(nt_pier,1,1))],
                                                                 [topo_dune['elevation'],
                                                                  topo_pier['elevation']],
                                                                 XX,YY,method=interp_method)

In [ ]:
plt.pcolor(XX,YY,Z_all,shading='auto')
plt.colorbar()

In [ ]:
interact(lambda ii: plt.plot(XX[ii,:],Z_all[ii,:],'b'),ii=(0,XX.shape[0]-1))

## Extend merged topo in alongshore direction

In [ ]:
Y_start_index,Y_end_index,Z_tmp=dut.extend_alongshore(XX,YY,Z_all)

In [ ]:
interact(lambda ii: plt.plot(XX[ii,:],Z_tmp[ii,:],'o'),ii=(0,XX.shape[1]))

In [ ]:
plt.pcolor(XX,YY,Z_tmp,shading='auto')
plt.colorbar()
plt.xlim([0,200])

## Load bathymetry transects

In [ ]:
#times out
repull=False
if repull:
    bathy_data = go.getBathyTransectFromNC()#(profilenumbers=960)
    pickle.dump(bathy_data, open(os.path.join(datacache,"bathy_data_{0}.p".format(d1.date())), "wb" ) )


In [ ]:
if 'bathy_data' not in globals():
    bathy_data=pickle.load(open(os.path.join(datacache,"bathy_data_{0}.p".format(d1.date())),"rb"))

In [ ]:
bathy_data.keys()

In [ ]:
bathy_data['elevation'].shape,bathy_data['xFRF'].shape

In [ ]:
bathy_points=np.vstack((bathy_data['xFRF'],bathy_data['yFRF'])).T

In [ ]:
plt.scatter(bathy_points[:,0],bathy_points[:,1],c=bathy_data['elevation'])

In [ ]:
points=np.vstack((all_points,bathy_points))
values=np.concatenate((all_values,bathy_data['elevation']))

In [ ]:
## try to evaluate accuracy on a few points
check_points=np.empty(shape=(0,2))
check_values=np.empty(shape=(0,))
if check_error:
    check_points,check_values,new_points,new_values,check_inds=dut.extract_values_and_points(points,values,cross_check_fraction)
    orig_points=points.copy()
    orig_values=values.copy()
    points=new_points
    values=new_values


In [ ]:
print('Total number of data points is {0}'.format(points.shape[0]))

In [ ]:
Z_interp=scipy.interpolate.griddata(points,values,(XX,YY),method=interp_method,fill_value=np.nan)

In [ ]:
plt.pcolor(XX,YY,Z_interp,shading='auto')

In [ ]:
Y_start_index,Y_end_index,Z_gridded=dut.extend_alongshore(XX,YY,Z_interp)

In [ ]:
interact(lambda ii: plt.plot(XX[ii,:],Z_gridded[ii,:],'-'),ii=(0,XX.shape[1]))

In [ ]:
plt.pcolor(XX,YY,Z_gridded,shading='auto')

## Check the error

In [ ]:
RMSE=None
if check_error:
    Z_check=scipy.interpolate.griddata(points,values,check_points,method=interp_method,fill_value=np.nan)
    diff=check_values-Z_check
    RMSE=np.sqrt(np.sum(diff**2)/float(diff.size))
    print('RMSE on {0}% random points is {1}'.format(cross_check_fraction,RMSE))


In [ ]:
fig,plotname=dut.plot_bathy2d_with_obs(XX,YY,Z_gridded,points,cross_check_fraction,RMSE,d1.date(),plotdir=os.path.curdir,
                          bathyTime=None, topoDuneTime=None, topoPierTime=None)
fig

## Write out the gridded bathy

### setup coordinates

In [ ]:
# get position stuff that will be constant for all surveys!!!
xFRFi_vecN = XX.reshape((1, XX.shape[0] * XX.shape[1]))[0]
yFRFi_vecN = YY.reshape((1, YY.shape[0] * YY.shape[1]))[0]
# convert FRF coords to lat/lon
test = geoprocess.FRF2ncsp(xFRFi_vecN, yFRFi_vecN)
# go through stateplane to avoid FRFcoords trying to guess the input coordinate systems
temp = geoprocess.ncsp2LatLon(test['StateplaneE'], test['StateplaneN'])
lat_vec = temp['lat']
lon_vec = temp['lon']
E_vec = test['StateplaneE']
N_vec = test['StateplaneN']

latitude = lat_vec.reshape(XX.shape[0], XX.shape[1])
longitude= lon_vec.reshape(XX.shape[0], XX.shape[1])
easting  = E_vec.reshape(XX.shape[0], XX.shape[1])
northing = N_vec.reshape(XX.shape[0], XX.shape[1])

xFRF = XX[0, :]
yFRF = YY[:, 1]


In [ ]:
outfile='IntegratedBathyTopo-{0}.nc'.format(d1.date())

In [ ]:
nc_dict={}
nc_dict['elevation'] = Z_gridded[np.newaxis,:,:]
nc_dict['xFRF'] = xFRF
nc_dict['yFRF'] = yFRF
nc_dict['latitude'] = latitude
nc_dict['longitude'] = longitude
nc_dict['northing'] = northing
nc_dict['easting'] = easting
nc_dict['updateTime'] = np.ones_like(nc_dict['elevation'])*d1.timestamp()
nc_dict['time'] = d1.timestamp()
nc_dict['survey_time'] = np.nanmean(bathy_data['epochtime'])
nc_dict['error_fraction'] = cross_check_fraction
nc_dict['error_estimate'] = RMSE

In [ ]:
py2netCDF.makenc_generic(outfile,global_yaml,var_yaml,nc_dict)

In [ ]:
foo=netCDF4.Dataset(outfile)

In [ ]:
foo

In [ ]:
np.nanmax(np.absolute(foo['elevation'][0]-Z_gridded))

In [ ]:
foo['survey_time'][:]

In [ ]:
tmpx,tmpy=np.meshgrid(foo['xFRF'],foo['yFRF'])
tmpz=foo['elevation'][0]


In [ ]:
foo.close()

In [ ]:
plt.pcolor(tmpx,tmpy,tmpz,shading='auto')